## 3.2.2 Logistic Regression Example with Word2Vec

### Word2Vec Feature Example

In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
# 판다스 데이터프레임으로 받아오기
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
# 리뷰값, 라벨값 리스트로 저장
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [5]:
# 텍스트 데이터를 각 단어들의 리스트로 나눈다.
# word2vec의 경우 단어로 표현된 리스트를 입력값으로 넣어야 하기 때문
sentences = []
for review in reviews:
    sentences.append(review.split())

In [6]:
# 학습시 필요한 파라미터

# 워드 벡터 특징값 수
num_features = 300   
# 단어에 대한 최소 빈도수
min_word_count = 40   
# 프로세스 개수
num_workers = 4
# 컨텍스트 윈도우 크기
context = 10          
# 다운 샘플링 비율
downsampling = 1e-3

# num_features: 각 단어에 대해 임베딩된 벡터의 차원을 정한다
# min_word_count: 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다.
# num_workers: 모델 학습 시 학습을 위한 프로세스 개수를 지정한다.
# context: word2vec을 수행하기 위한 컨텍스트 윈도우 크기를 지정한다.
# downsampling: word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율을 지정한다. 보통 0.001이 좋은 성능을 낸다고 한다.

In [7]:
# 학습 진행 상황 확인
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [8]:
from gensim.models import word2vec

# word2vec객체 생성
model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2020-07-23 17:27:24,963 : INFO : collecting all words and their counts
2020-07-23 17:27:24,966 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-23 17:27:25,180 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-07-23 17:27:25,401 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-07-23 17:27:25,511 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-07-23 17:27:25,512 : INFO : Loading a fresh vocabulary
2020-07-23 17:27:25,550 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-07-23 17:27:25,551 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-07-23 17:27:25,572 : INFO : deleting the raw counts dictionary of 74065 items
2020-07-23 17:27:25,574 : INFO : sample=0.001 downsamples 30 most-common words
2020-07-23 17:27:25,574 :

In [9]:
# 문장의 특징 값을 만드는 함수
# 리뷰마다 단어의 개수가 모두 다르기 때문에 하나의 형태로 만들어줘야 하기 때문

# word: 단어의 모음인 하나의 리뷰가 들어간다.
# model: word2vec 모델을 넣는 곳이며, 우리가 학습한 word2vec 모델이 들어간다.
# num_features: word2vec으로 임베딩할 때 정했던 벡터의 차원 수를 뜻한다.
def get_features(words, model, num_features):
    # 출력 벡터 초기화(속도를 빠르게 하기 위함)
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            # 사전에 해당하는 벡터를 더함
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])
    
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [10]:
# 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수

# reviews: 학습 데이터인 전체 리뷰 데이터를 입력하는 인자
# model: word2vec 모델을 입력하는 인자. 위 함수와 같이 앞에서 학습시킨 모델을 넣는다.
# num_features: word2vec으로 임베딩할 때 정했던 벡터의 차원 수를 뜻한다.
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [11]:
test_data_vecs = get_dataset(sentences, model, num_features)

<ipython-input-9-48cdc3edcd51>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


In [12]:
# 학습, 검증 데이터 분리
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [13]:
# 모델 선언, 학습
from sklearn.linear_model import LogisticRegression


lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

C:\Users\jch\anaconda3\envs\pr_tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [14]:
# 결과 확인
predicted = lgs.predict(X_test)
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

------------
Accuracy: 0.863400
Precision: 0.858594
Recall: 0.872568
F1-Score: 0.865525
AUC: 0.933716


In [15]:
# 데이터 제출
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

# 리뷰를 담은 리스트 생성
test_review = list(test_data['review'])

In [16]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [17]:
# 리뷰의 단어의 리스트로 빈 리스트 생성
test_sentences = list()

# 빈 리스트에 리뷰의 단어를 넣는다
for review in test_review:
    test_sentences.append(review.split())

In [18]:
# 리뷰의 단어에 word2vec적용
test_data_vecs = get_dataset(test_sentences, model, num_features)

<ipython-input-9-48cdc3edcd51>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


In [19]:
# 선형 회귀 모델로 예측
test_predicted = lgs.predict(test_data_vecs)

In [20]:
ids = list(test_data['id'])

# id값과 예측 결과값을 판다스 데이터프레임으로 만듦
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [21]:
# 파일로 저장
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)